# Call Google Search with tool use

## Learning goals

* Learn tool use with JSON output
* Give your LLM an option to search Google if it cannot answer the question

Put stuff here

***

In [1]:
%pip install -qU pip
%pip install requirements.txt

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement requirements-new.txt (from versions: none)
ERROR: No matching distribution found for requirements-new.txt
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import json
import os
import shutil
import requests
import string
from googlesearch import search
from bs4 import BeautifulSoup
from datetime import datetime
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

In [2]:
#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'
modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0'

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

***

## Google example

put stuff here

In [3]:
def get_page_content(url):
    try:
        response = requests.get(url)
        if response:
            # Parse HTML content
            soup = BeautifulSoup(response.text, 'html.parser')
            # Remove script and style elements
            for script_or_style in soup(["script", "style"]):
                script_or_style.decompose()
            # Get text
            text = soup.get_text()
            # Break into lines and remove leading and trailing space on each
            lines = (line.strip() for line in text.splitlines())
            # Break multi-headlines into a line each
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            # Drop blank lines
            cleaned_text = '\n'.join(chunk for chunk in chunks if chunk)
            return cleaned_text
        else:
            raise Exception("No response from the server.")
    except Exception as e:
        print(f"Error while fetching and cleaning content from {url}: {e}")
        return None

def search_google(query):
    try:
        search_results = []
        for j in search(query, sleep_interval=5, num_results=3):
            search_results.append(j)
        return search_results
    except Exception as e:
        print(f"Error during Google search: {e}")
        return []

def handle_search(event):
    #input_text = event.get('inputText', '')  # Extract 'inputText'
    input_text = event

    # Proceed with Google search
    print("Performing Google search...")
    urls_to_scrape = search_google(input_text)

    aggregated_content = ""
    results = ""
    for url in urls_to_scrape:
        #print("URLs Used: ", url)
        content = get_page_content(url)
        if content:
            #print("CONTENT: ", content)
            #filename = url.split('//')[-1].replace('/', '_') + '.txt'  # Simple filename from URL
            #aggregated_content += f"URL: {url}\n\n{content}\n\n{'='*100}\n\n"
            #results.append({'url': url, 'status': 'Content aggregated'})
            results = results + content

    return results



In [4]:
#tool definition

toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "google_search",
        "description": "A tool to retrieve an up to date information from a Google search.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "question": {
                "type": "string",
                "description": "The users question as-is to be searched by Google"
              }
            },
            "required": ["question"]
          }
        }
      }
    }
  ]
}

In [5]:
#The function that answers the question based on google search content
def answer_question_with_content(question, content):
    query = f"""
    Based solely on this content:
    <content>
    {content}
    </content>
    Answer this question:
    <question>
    {question}
    </question>
    Skip any preamble or references to the tool.
    """

    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "system": [
            { "text": "You are an expert research assistant." }
            ],
        "inferenceConfig": {
            "maxTokens": 4096,
            "temperature": 0
        }
    }

    response = bedrock_client.converse(**converse_api_params)
    #print(f"\nresponse = {response}\n")
    return response

In [6]:
#The function that generates the json for a given article subject
def answer_question(question):
    query = f"""
    <question>
    {question}
    </question>

    You have access to the google_search tool. Only use the google_search tool if you cannot answer the question from your knowledge. 
    For example only use the tool if the subject or event is too new.
    Skip the preamble.
    
    """

    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "toolConfig": toolConfig,
        "system": [
            { "text": "You are an expert research assistant." }
            ],
        "inferenceConfig": {
            "maxTokens": 4096,
            "temperature": 0
        }
    }

    response = bedrock_client.converse(**converse_api_params)
    #print(f"\nresponse = {response}\n")

    google_search = None
    for content in response['output']['message']['content']:
        if isinstance(content, dict) and 'toolUse' in content:
            tool_use = content['toolUse']
            if tool_use['name'] == "google_search":
                google_search = tool_use['input']
                break

    if google_search:
        #print(json.dumps(google_search, indent=2))
        question = google_search["question"]
        content = handle_search(question)
        if content:
            print("Google search successful")
            response = answer_question_with_content(question, content)
            print(f"\nresponse = {response['output']['message']['content'][-1]['text']}\n")
        else:
            print("No content found from Google search")
    else:
        print("No Google search found in the response.")
        print(f"\nresponse = {response['output']['message']['content'][-1]['text']}\n")

In [7]:
answer_question("Who won the 2019 Masters golf tournament?")

No Google search found in the response.

response = Based on my knowledge, I can answer this question without using the google_search tool.

Tiger Woods won the 2019 Masters golf tournament. This was a significant victory for Woods, as it marked his first major championship win in 11 years and his fifth Masters title overall. The tournament took place from April 11-14, 2019, at Augusta National Golf Club in Augusta, Georgia. Woods finished with a score of 13 under par, one stroke ahead of his closest competitors.



In [8]:
answer_question("Who won the 2023 Masters golf tournament?")

Performing Google search...
Google search successful

response = Jon Rahm won the 2023 Masters golf tournament. He shot a 12-under 276 for the weekend, beating out Phil Mickelson and Brooks Koepka who tied for second at 8-under 280.



In [9]:
answer_question("What is the current weather in Seattle, Wa right now?")

Performing Google search...
Google search successful

response = According to the weather information provided, the current weather in Seattle, WA is:

68°F, Mostly Cloudy, with winds from the North at 3 mph gusting to 5 mph.



In [11]:
answer_question("What is the current time and date in Seattle, WA?")

No Google search found in the response.

response = I can answer this question without using the google_search tool, as the current time and date in Seattle, WA is not information that requires an up-to-date search.

Seattle, WA is in the Pacific Time Zone (PT). The current time and date in Seattle would depend on when this response is being read. However, I can provide you with the general time zone information:

Seattle follows Pacific Time (PT), which is typically:
- Pacific Standard Time (PST) during the winter months
- Pacific Daylight Time (PDT) during the summer months (when daylight saving time is in effect)

Seattle is usually 8 hours behind Coordinated Universal Time (UTC-8) during standard time, and 7 hours behind (UTC-7) during daylight saving time.

To get the exact current time and date in Seattle, you would need to check a real-time clock or use a device set to the Pacific Time Zone.



***